In [ ]:
#Mount Google Drive with this Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy
from astropy.io import fits
from astropy.visualization import AsinhStretch, MinMaxInterval
from PIL import Image, ImageOps
from tqdm import tqdm
import glob
import os

In [ ]:
def arcsinh_stretch_ds9(data, a=10, factor=3):
    #https://ds9.si.edu/doc/ref/how.html
    return np.arcsinh(data * a)/factor

In [ ]:
def create_jpg(img,path_jpg,name,target_size=300):
    native_pil_image = Image.fromarray((img * 255).astype(np.uint8),mode='L').convert('RGB')
    resized_image = native_pil_image.resize(size=(target_size, target_size), resample=Image.LANCZOS)
    resized_image_flip = ImageOps.flip(resized_image)
    resized_image_flip.save(path_jpg+name+".jpg")

In [ ]:
def convert_to_jpgs(path_fits, path_jpgs, target_size=300):
    print("Converting to jpgs...")
    filenames = glob.glob(path_fits+"*.fits")
    if not os.path.exists(path_jpgs):
        os.makedirs(path_jpgs)
    interval = MinMaxInterval()
    failures = []
    for i in tqdm(range(len(filenames))):
        try:
            image_path = filenames[i]
            filenames[i] = filenames[i].replace(path_fits,"").replace(".fits","")
            image_data = fits.getdata(image_path, ext=0)
            image_data_norm = interval(image_data)
            image_data_asinh = arcsinh_stretch_ds9(image_data_norm)
            create_jpg(image_data_asinh,path_jpgs,filenames[i],target_size)
        except Exception as e:
            print("Error:",e)
            failures.append(filenames[i])

In [ ]:
!unzip /content/drive/MyDrive/MPE/ZoobotEuclid/Data/Euclidized_COSMOS_complete/fits_euclid_complete.zip -d /content/fits_euclid_complete

In [ ]:
convert_to_jpgs("fits_euclid_complete/fits_euclid/","jpgs_euclid_complete/")

In [ ]:
!zip -r /content/jpgs_euclid_complete.zip jpgs_euclid_complete

In [ ]:
!cp jpgs_euclid_complete.zip /content/drive/MyDrive/MPE/ZoobotEuclid/Data/Euclidized_COSMOS_complete/